### Imports

In [1]:
import glob
import cv2

### Functions

In [19]:
def getbboxesfromgtfile(file):
    filename = file.split('/')[-1:][0]
    f = open(file, 'r')
    #imgfile = gtimages + filename[:-4] + '.jpg'
    #print(imgfile)
    #img = cv2.imread(imgfile)
    #height, width, channel = img.shape
    result = []
    for lines in f:
        llist = lines.split(' ')
        x0 = int(llist[1])
        y0 = int(llist[2])
        #For cage gt, we will have to add x0 and y0
        x1 = x0 + int(llist[3])
        y1 = y0 + int(llist[4])
        label = llist[0]
        bbox = [x0, y0, x1, y1]
        bboxdata = [label, bbox]
        #print(bboxdata)
        result.append(bboxdata)
    return result

def getbboxesfromdettfile(file):
    f = open(file, 'r')
    result  = []
    i = 0
    for lines in f:
        if i == 0:
            i = 1
            continue
        llist = lines.split(' ')
        x0 = int(llist[2])
        y0 = int(llist[3])
        x1 = int(llist[4]) + x0
        y1 = int(llist[5]) + y0
        label = 'Unassigned'
        conf = float(llist[1])
        bbox = [x0, y0, x1, y1]
        bboxdata = [label, bbox, conf]
        #print(bboxdata)
        result.append(bboxdata)
    return result

def filterbboxesforclass(bboxdata, classname):
    result = []
    for b in bboxdata:
        if b[0] == classname:
            result.append(b)
    return result

def iou(boxA, boxB):
    # if boxes dont intersect
    if boxesIntersect(boxA, boxB) is False:
        return 0
    interArea = getIntersectionArea(boxA, boxB)
    union = getUnionAreas(boxA, boxB, interArea=interArea)
    # intersection over union
    iou = interArea / union
    assert iou >= 0
    return iou

# boxA = (Ax1,Ay1,Ax2,Ay2)
# boxB = (Bx1,By1,Bx2,By2)
def boxesIntersect(boxA, boxB):
    if boxA[0] > boxB[2]:
        return False  # boxA is right of boxB
    if boxB[0] > boxA[2]:
        return False  # boxA is left of boxB
    if boxA[3] < boxB[1]:
        return False  # boxA is above boxB
    if boxA[1] > boxB[3]:
        return False  # boxA is below boxB
    return True

def getArea(box):
    return (box[2] - box[0] + 1) * (box[3] - box[1] + 1)


def getIntersectionArea(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # intersection area
    return (xB - xA + 1) * (yB - yA + 1)


def getUnionAreas(boxA, boxB, interArea=None):
    area_A = getArea(boxA)
    area_B = getArea(boxB)
    if interArea is None:
        interArea = getIntersectionArea(boxA, boxB)
    return float(area_A + area_B - interArea)

def createdetfiles(file, detections):
    writeline = []
    for d in detections:
        line = d[0] + ' ' + str(d[2]) + ' ' + str(d[1][0]) + ' ' + str(d[1][1])  + ' ' + str(d[1][2] - d[1][0])  + ' ' + str(d[1][3] - d[1][1]) + '\n' 
        #print(line)
        writeline.append(line)
    f = open(file, 'w+')
    f.writelines(writeline)

### Ground Truth Files

In [12]:
rootpath = '../../temp/'
path = rootpath + 'txt_gt/*txt'
imgpath = '../../data/00_docbank_20k/images/'
files = glob.glob(path)
print(len(files))
### Classwise Details
for file in files:
    f = open(file, 'r')
    filename = file.split('/')[-1:][0]
    g = open(rootpath + 'txt_gt_classwise/' + filename , 'w+')
    imgfile = imgpath + filename[:-4] + '_ori.jpg'
    img = cv2.imread(imgfile)
    height, width, channel = img.shape
    lines = []
    for line in f:
        line.replace("\t", ' ')
        try:
            llist = line.split(' ')
            x0 = int(int(llist[1]) * width / 1000)
            y0 = int(int(llist[2]) * height / 1000)
            x1 = int(int(llist[3]) * width / 1000)
            y1 = int(int(llist[4]) * height / 1000)
            label = llist[-1][:-1]
            writeline = label + ' ' + str(x0) +  ' ' + str(y0) +  ' ' + str(x1 - x0) +  ' ' + str(y1 - y0) + '\n'
            if 'figure' not in label:
                lines.append(writeline)
        except:
            print(file)
            print(line)
    g.writelines(lines)
    f.close()
    g.close()
    
    
### Saving Results
# for i in range(100):
#     filename = files[i].split('/')[-1:][0]
#     print(filename)
#     imgfile = files[i][:-4] + '_ori.jpg'
#     img = cv2.imread(imgfile)
#     img = 0 * img
#     img = ~img
#     g = open('groundtruth/' + filename , 'r')
#     for line in g:
#         llist = line.split(' ')
#         x0 = int(llist[1])
#         y0 = int(llist[2])
#         w = int(llist[3])
#         h = int(llist[4])
#         fimg = cv2.rectangle(img, (x0, y0), (x0 + w, y0 + h), (255,0,255), cv2.FILLED)
#     cv2.imwrite('results/' + filename[:-4] + '.jpg' , fimg)

3465
../../temp/txt_gt/44.tar_1404.2296.gz_fntn_1.txt
"sign	787	723	822	737	0	0	0	PSDFFF+NimbusRomNo9L-Regu	paragraph

../../temp/txt_gt/44.tar_1404.2296.gz_fntn_1.txt
"optimal	321	778	379	792	0	0	0	PSDFFF+NimbusRomNo9L-Regu	paragraph

../../temp/txt_gt/43.tar_1802.05986.gz_SW_arXiv_submission_1.txt
"broad	720	741	773	759	0	0	0	WDJGRP+SFRM1095	paragraph

../../temp/txt_gt/47.tar_1404.3841.gz_paper_3d_desy_arxiv_3.txt
"night	835	792	875	807	0	0	0	VYJYOQ+NimbusRomNo9L-Regu	paragraph

../../temp/txt_gt/47.tar_1404.3841.gz_paper_3d_desy_arxiv_3.txt
sky-	879	792	907	807	0	0	0	VYJYOQ+NimbusRomNo9L-Regu	paragraph

../../temp/txt_gt/47.tar_1404.3841.gz_paper_3d_desy_arxiv_3.txt
the	92	806	112	821	0	0	0	VYJYOQ+NimbusRomNo9L-Regu	paragraph

../../temp/txt_gt/47.tar_1404.3841.gz_paper_3d_desy_arxiv_3.txt
stopping	117	806	173	821	0	0	0	VYJYOQ+NimbusRomNo9L-Regu	paragraph

../../temp/txt_gt/47.tar_1404.3841.gz_paper_3d_desy_arxiv_3.txt
power	179	806	219	821	0	0	0	VYJYOQ+NimbusRomNo9L-Regu	paragraph

In [ ]:
for i in range(100):
    filename = files[i].split('/')[-1:][0]
    print(filename)
    imgfile = files[i]
    txtfile = filename[:-4] + '.txt'
    img = cv2.imread(imgfile)
    #img = 0 * img
    #img = ~img
    g = open('doctr_detections/' + txtfile , 'r')
    for line in g:
        llist = line.split(' ')
        x0 = int(llist[2])
        y0 = int(llist[3])
        w = int(llist[4])
        h = int(llist[5])
        fimg = cv2.rectangle(img, (x0, y0), (x0 + w, y0 + h), (0,255,0), 2)
    cv2.imwrite('resultswithdetections/' + filename[:-4] + '.jpg' , fimg)

## CAGE Associate

In [22]:
gtpath = '../../temp/txt_gt_classwise/*.txt'
gtimages = '../../data/00_docbank_20k/images/'
detpath = '../../temp/txt_pred/*.txt'
outdetpath = '../../temp/txt_pred_classwise/'
gtfiles = glob.glob(gtpath)
detfiles = glob.glob(detpath)
print(len(gtfiles))
print(len(detfiles))

3465
3465


In [ ]:
for i in range(len(gtfiles)):
    filename = gtfiles[i].split('/')[-1:][0]
    groundtruths = getbboxesfromgtfile(gtfiles[i])
    detections = getbboxesfromdettfile(detfiles[i])
    gts = groundtruths
    #gts = filterbboxesforclass(groundtruths, classname)

    for d in detections:
        iou_max = 0
        for g in gts:
            detbox = d[1]
            gtbox = g[1]
            #print(detbox)
            #print(gtbox)
            iou_candidate = iou(gtbox, detbox)
            #if iou_candidate >= iou_thresh:
            if iou_candidate >= iou_max:
                iou_max = iou_candidate
                d[0] = g[0]
    outputfile = outdetpath + filename
    createdetfiles(outputfile, detections)

## DocTR Associate

In [ ]:
gtpath = '../../docbank_processed/gt_cage_sample100/gt_cage_classwise/*.txt'
gtimages = '../../docbank_processed/processed_data/spear_ori_black/'
detpath = '../../docbank_processed/processed_data/out_txt_8/*.txt'
outdetpath = '../../docbank_processed/processed_data/out_txt_8_classwise/'
gtfiles = glob.glob(gtpath)
detfiles = glob.glob(detpath)

for i in range(len(gtfiles)):
    filename = gtfiles[i].split('/')[-1:][0]
    groundtruths = getbboxesfromgtfile(gtfiles[i])
    detections = getbboxesfromdettfile(detfiles[i])
    gts = groundtruths
    #gts = filterbboxesforclass(groundtruths, classname)

    for d in detections:
        iou_max = 0
        for g in gts:
            detbox = d[1]
            gtbox = g[1]
            #print(detbox)
            #print(gtbox)
            iou_candidate = iou(gtbox, detbox)
            #if iou_candidate >= iou_thresh:
            if iou_candidate >= iou_max:
                iou_max = iou_candidate
                d[0] = g[0]
    outputfile = outdetpath + filename
    createdetfiles(outputfile, detections)